In [2]:
import pandas as pd
df = pd.read_csv('..\data\Audi_bmw_100posts_full.csv')
df.head(15)

,type,post_id,post_title,comment_id,author,body,score,created_utc,parent_id
0,post,1q4ia7n,First Time Audi Owner,NaN,Colgray21,Traded up from a BMW 535i to this 2024 Q5 and ...,15,1767609078.0,NaN
1,comment,1q4ia7n,First Time Audi Owner,nxtjblw,ecom_ryan,Welcome to the club!,1,2026-01-05 15:16:56,t3_1q4ia7n
2,post,1q3wjtk,Talk me out of it: 2016 A5 Sportback 3.0T (B8....,NaN,ThatOtherOmar,I’m considering trading my 2007 BMW E92 328i f...,18,1767549310.0,NaN
3,comment,1q3wjtk,Talk me out of it: 2016 A5 Sportback 3.0T (B8....,nxnvmrx,Beemeristic,"Easy, if you don't have maintenance/repair mon...",9,2026-01-04 18:59:19,t3_1q3wjtk
4,comment,1q3wjtk,Talk me out of it: 2016 A5 Sportback 3.0T (B8....,nxrvusx,PurpleSlightlyRed,Previous owner spent more money on the fake RS...,1,2026-01-05 07:20:05,t1_nxnvmrx
5,comment,1q3wjtk,Talk me out of it: 2016 A5 Sportback 3.0T (B8....,nxo011u,alexberbo,"Skip, those mods are terrible, fake RS bumper ...",10,2026-01-04 19:18:35,t3_1q3wjtk
6,comment,1q3wjtk,Talk me out of it: 2016 A5 Sportback 3.0T (B8....,nxnwaxy,96JY,The seats are a shade of poop.,4,2026-01-04 19:02:22,t3_1q3wjtk
7,comment,1q3wjtk,Talk me out of it: 2016 A5 Sportback 3.0T (B8....,nxosn9l,No-Room-3886,I didnt know these existed. Previous dudes don...,3,2026-01-04 21:25:44,t3_1q3wjtk
8,comment,1q3wjtk,Talk me out of it: 2016 A5 Sportback 3.0T (B8....,nxp1gmt,xRaffaell,Probably fake kms or not mentained properly du...,2,2026-01-04 22:06:40,t3_1q3wjtk
9,comment,1q3wjtk,Talk me out of it: 2016 A5 Sportback 3.0T (B8....,nxp3m59,Super-Total-661,You dont need us to talk you out of it or into...,2,2026-01-04 22:16:41,t3_1q3wjtk


# Dataset first impressions
- distinction between posts and comments. A comment can be on a post or on another comment
- Score seems to rank the Posts somehow
- There is no Unique ID for each post! Comment ID seems to be the Unique ID for all the posts => Do we need to merge it somehow for posts? => if type post then ... else ...
- created_utc is sometimes stored as "Unix epoch Time" and sometimes as human-readable timestamp => needs to be converted to pd.DateTime, no type was inferred => seems to apply to posts only
- Some text (post_title and body)contains emojis => need to be removed e.g. CommentID = ne2vag2
- Text contains newline signs (/n) => need to be removed as well

In [3]:
# Create single unique id
df['id'] = df['comment_id'].fillna(df['post_id'])

# Check if it is unique
if df['id'].nunique() != df['id'].count():
    raise ValueError('Found identifiers that are not unique')

# Drop old IDs
df.drop('comment_id', axis=1, inplace=True)
df.drop('post_id', axis=1, inplace=True)

In [4]:
# Check if any Values are still NaN
df.isna().sum()

type             0
post_title       0
author           0
body             5
score            0
created_utc      0
parent_id      100
id               0
dtype: int64

In [5]:
# Looks like Some posts have a title but no content in body
df[df['body'].isna()]

,type,post_title,author,body,score,created_utc,parent_id,id
664,post,"Audi RS3 vs BMW M2/M3 steering wheel, which is...",Substantial_Copy_576,NaN,543,1765489803.0,NaN,1pka2tt
1101,post,Audi Quattro vs BMW xDrive vs Mercedes 4MATIC ...,1mrwick,NaN,11,1763822923.0,NaN,1p3v16z
1311,post,Hey guys is this device good? I've never used ...,Icy-Finger-1502,NaN,2,1763231736.0,NaN,1oxz7wa
2831,post,🚨 2026 Audi Prototypes in Tandem – Chased by B...,markymark666full,NaN,2,1757087350.0,NaN,1n98vdx
3744,post,"Audi’s New Q5 Is Slick, but the SQ5 Is Built t...",Simple-Web2502,NaN,50,1753872072.0,NaN,1md37cy


In [6]:
# Since 5 posts (5% of all posts) have no body dropping them is not feasible, we use the post_title twice for those instances
df['body'] = df['body'].fillna(df['post_title'])
df.isna().sum() # Now all the samples have body content

type             0
post_title       0
author           0
body             0
score            0
created_utc      0
parent_id      100
id               0
dtype: int64

In [7]:
# Unicode emoji range: https://stackoverflow.com/questions/72477087/how-to-select-only-rows-containing-emojis-and-emoticons-in-python
df[df['body'].str.contains(r'[\u263a-\U0001f645]') | df['post_title'].str.contains(r'[\u263a-\U0001f645]')]

,type,post_title,author,body,score,created_utc,parent_id,id
10,comment,Talk me out of it: 2016 A5 Sportback 3.0T (B8....,Important-Ad-6754,My guess is your Lebanese 👌,2,2026-01-04 22:25:35,t3_1q3wjtk,nxp5j4c
16,post,Don't buy the new A5. I did.,saintmarko,Just dropped 65k € for a new A5 2.0 TDI.\n\nTh...,765,1767551128.0,NaN,1q3xd30
36,comment,Don't buy the new A5. I did.,Silverbullets24,The snow performance is wild to me… I had a 20...,4,2026-01-04 22:54:42,t1_nxodhpd,nxpbubj
84,comment,Don't buy the new A5. I did.,saintmarko,Okay 🤷🏻‍♂️,2,2026-01-04 23:11:16,t1_nxpe5ce,nxpfeb9
94,comment,Don't buy the new A5. I did.,Nurse_Neurodiver3176,Getting rid of the A4 is going to be a costly ...,3,2026-01-05 06:47:02,t3_1q3xd30,nxrrmv4
...,...,...,...,...,...,...,...,...
4955,comment,Audi is in big trouble,Alarming-Run7175,I contacted the dealership up here ready to or...,50,2025-07-04 01:21:02,t1_n17kzna,n17rocs
5012,comment,Audi is in big trouble,Snowvid2021,This ✌🏻,2,2025-07-04 04:39:25,t1_n1835oh,n18nw7d
5086,comment,Audi is in big trouble,Sudden-Breadfruit-99,"I’m an Audi purist, with the new lineup this i...",2,2025-07-04 23:43:26,t3_1lr2jt5,n1djm40
5109,comment,Audi is in big trouble,Quags_77,So they got rid of the A4 this year (their wor...,1,2025-07-04 03:43:07,t3_1lr2jt5,n18ew3w


In [8]:
# Remove emojis from, also see: https://stackoverflow.com/questions/57514169/how-can-i-remove-emojis-from-a-dataframe
df[['body', 'post_title']] = (df[['body', 'post_title']]
 .astype(str)
 .apply(lambda s: s.str.encode('ascii', 'ignore') # 'ignore' ignores the characters that cannot be encoded => emojis
                    .str.decode('ascii')))

In [9]:
# Text contains newline signs. To avoid sticking words together \n`s are replaced by single whitespace
df['body'] = df['body'].str.replace(r'\n+', ' ', regex=True)

In [10]:
# created_utc is sometimes stored as "Unix epoch Time" and sometimes as human-readable timestamp => needs to be converted to pd.DateTime, no type was inferred => seems to apply to posts only
posts = df['type'] == 'post'
df.loc[posts, 'created_utc'] = pd.to_datetime(df.loc[posts, 'created_utc'], unit='s')

C:\Users\micha\AppData\Local\Temp\ipykernel_15704\3368709646.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df.loc[posts, 'created_utc'] = pd.to_datetime(df.loc[posts, 'created_utc'], unit='s')


# Create Sentiment Features

In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

sentiment_cols = df['body'].apply(lambda bod: pd.Series(analyzer.polarity_scores(bod)))

df = pd.concat([df, sentiment_cols], axis=1)

df['sentiment_type'] = 'Neutral' # Default Neutral
df.loc[df['compound'] > 0.05, 'sentiment_type'] = 'Positive'
df.loc[df['compound'] < -0.05, 'sentiment_type'] = 'Negative'

# Find Swearwords

In [13]:
from better_profanity import profanity

profanity.load_censor_words()

df['swearwords'] =  df['body'].apply(lambda b: [word for word in b.split() if profanity.contains_profanity(word)])

# Save Prepared Dataframe

In [14]:
df.to_csv('../data/Audi_bmw_100posts_cleaned.csv', index=False)